<div dir="auto" align="center">
    <h3>
        بسم الله الرحمن الرحیم
    </h3>
    <br>
    <h1>
        <strong>
            بازیابی پیشرفته اطلاعات
        </strong>
    </h1>
    <h2>
        <strong>
            تمرین سوم (موتور جستجوی اخبار)
        </strong>
    </h2>
    <br>
    <h3>
        محمد هجری - ٩٨١٠٦١٥٦
        <br><br>
        ارشان دلیلی - ٩٨١٠٥٧٥١
        <br><br>
        سروش جهان‌زاد - ٩٨١٠٠٣٨٩
    </h3>
    <br>
</div>
<hr>

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        دسترسی به داده‌ها و مدل‌های ذخیره شده
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        به دلیل حجم بالای فایل‌های ذخیره شده، از قرار دادن آن‌ها در پوشه نوتبوک صرف نظر کرده و با اجرای قطعه کد زیر، از طریق گوگل درایو به آن‌ها دسترسی پیدا می‌کنیم.
    </p>
</div>

In [ ]:
# import sys
# from google.colab import drive
# drive.mount('/content/drive')
# address = "University/Term 6/MIR/Homeworks/HW3"
# sys.path.append(f"/content/drive/My Drive/{address}")
# %cd /content/drive/My\ Drive/$address

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        نصب و دسترسی به کتابخانه‌های مورد نیاز
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای دو قطعه کد زیر، کتابخانه‌هایی که از آن‌ها در این تمرین استفاده شده است، نصب و قابل استفاده می‌شوند.
    </p>
</div>

In [ ]:
# !pip install bs4
# !pip install tqdm
# !pip install pandas
# !pip install requests
# !pip install hazm
# !pip install unidecode
# !pip install fasttext
# !pip install scikit-learn
# !pip install pandas
# !pip install nltk
# !pip install torch
# !pip install transformers
# !pip install sentence_transformers
# !pip install faiss-cpu
# !pip install faiss-gpu

In [ ]:
import os
import re
import csv
import hazm
import nltk
import json
import torch
import pickle
import zipfile
import requests
import fasttext
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from bs4 import BeautifulSoup
from string import punctuation
from IPython.display import display
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ١. دریافت داده‌ها
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این تمرین، بیش از ٦٨ هزار خبر از
        <a href="https://www.hamshahrionline.ir/"> وب‌سایت همشهری‌آنلاین </a>
        گردآوری شده که در ١٠ دسته‌ی سیاسی، جهانی، اقتصادی، اجتماعی، شهری، ورزشی، علمی، فرهنگی، فناوری اطلاعات و مهارت‌های زندگی طبقه‌بندی شده‌اند.
    </p>
</div>

In [ ]:
CATEGORIES = {
    'Politics': 'سیاسی',
    'World': 'جهانی',
    'Economy': 'اقتصادی',
    'Society': 'اجتماعی',
    'City': 'شهری',
    'Sport': 'ورزشی',
    'Science': 'علمی',
    'Culture': 'فرهنگی',
    'IT': 'فناوری اطلاعات',
    'LifeSkills': 'مهارت‌های زندگی',
}

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        برای دریافت داده‌ها یک ماژول Scraper ساخته‌ایم که اخبار مربوط به ١٠ دسته‌ی مذکور را در بازه‌ی زمانی تعیین شده، کراول کرده و در فایل dataset.zip ذخیره و فشرده سازی می‌کند. کد مربوط به این ماژول را در زیر مشاهده می‌کنید.
    </p>
</div>

In [ ]:
def read_dataset_from_file():
    dataset = []
    with zipfile.ZipFile('dataset.zip', 'r') as zip_file:
        zip_file.extractall()
    with open('dataset.csv', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)
        for row in csv_reader:
            data = dict(zip(header[1:], row[1:]))
            dataset.append(data)
    os.remove('dataset.csv')
    return dataset


dataset = pd.DataFrame(read_dataset_from_file())

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    با اجرای قطعه کد زیر، تعداد خبرهای هر دسته و تعداد کل خبرها را می‌توان مشاهده کرد.
    </p>
</div>

In [ ]:
def display_dataset_info():
    global CATEGORIES, dataset

    length_dict = {key: 0 for key in CATEGORIES.keys()}
    for _, data in dataset.iterrows():
        length_dict[data['category']] += 1

    df_dict = {
        'دسته': CATEGORIES.values(),
        'تعداد': length_dict.values(),
    }

    df = pd.DataFrame(df_dict)
    df.index += 1
    df.loc[0] = ['کل خبرها', len(dataset)]
    df = df.sort_index()
    display(df)


display_dataset_info()

<div>
    <h3 style='direction:rtl;text-align:justify;'>
        ٢. پیش پردازش اولیه‌ی متن
    </h3>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        ابزار مورد استفاده برای پیش‌پردازش متن ورودی به صورت ماژولار طراحی شده است؛ به طوری که با صدا زدن تابع preprocess از آن، متن داده شده با عبور از یک خط لوله به صورت مرحله به مرحله تغییر می‌کند تا به یک ساختار استاندارد برسد. این مراحل عبارتند از:
    </p>
</div>

<div dir="auto" align="justify">
    <li style='direction:rtl;text-align:justify;'>
        نرمال سازی داده‌ها (normalize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف لینک‌ها (remove_links)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف نشانه‌های نگارشی (remove_punctuations)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        واحد سازی داده‌ها (word_tokenize)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف کلمات نامعتبر (remove_invalid_words)
    </li>
    <li style='direction:rtl;text-align:justify;'>
        حذف ایست‌واژه‌ها (remove_stopwords)
    </li>
</div>

In [ ]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_invalid_words(words)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [ ]:
def save_preprocessed_texts(texts, path="Preprocessed_texts.pickle"):
    with open(path, "wb") as file:
        pickle.dump(texts, file)


def load_preprocessed_texts(path="Preprocessed_texts.pickle"):
    with open(path, "rb") as file:
        return pickle.load(file)


def data_to_text(data):
    return ' '.join([data['title']]).lower()


def get_preprocessed_texts(dataset, preprocessor, mode, save=False):
    preprocessed_texts = []
    if mode == 'process':
        texts = [data_to_text(data) for _, data in dataset.iterrows()]
        preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    if mode == 'load':
        preprocessed_texts = load_preprocessed_texts()
    if save:
        save_preprocessed_texts(preprocessed_texts)
    return preprocessed_texts

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از ماژول Preprocessor ایجاد کرده و شروع به پیش پردازش داده‌ها می‌کنیم، یا داده‌های پیش‌پردازش‌شده‌ی ذخیره‌شده را از فایل مربوطه بازیابی می‌کنیم.
    </p>
</div>

In [ ]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')
preprocessed_texts = get_preprocessed_texts(dataset, preprocessor, mode='load', save=False)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با توجه به این که حجم مجموعه داده‌ها باعث ایجاد محدودیت حافظه و زمان برای بعضی از مدل‌ها می‌شود، ناچار هستیم کسری از مجموعه‌ی داده‌ها را به عنوان ورودی برای یادگیری به آن‌ها بدهیم. با اجرای قطعه کد زیر، یک مجموعه داده‌ی کوچک‌تر از روی مجموعه داده‌ی اصلی ایجاد می‌کنیم.
    </p>
</div>

In [ ]:
def get_mini_dataset(len_each_category=400):
    global CATEGORIES, dataset

    mini_dataset = []
    for category in CATEGORIES.keys():
        dataset_by_category = dataset.loc[dataset['category'] == category]
        length = min(len_each_category, dataset_by_category.shape[0])
        mini_dataset.append(dataset_by_category.sample(length, random_state=1))

    mini_dataset = pd.concat(mini_dataset).reset_index(drop=True)
    texts = [data_to_text(data) for _, data in mini_dataset.iterrows()]
    mini_preprocessed_texts = [preprocessor.preprocess(text) for text in tqdm(texts)]
    return mini_dataset, mini_preprocessed_texts


mini_dataset, mini_preprocessed_texts = get_mini_dataset()

In [ ]:
vocabulary = set()

for doc in preprocessed_texts:
    vocabulary = vocabulary.union(set(doc))

vocabulary

In [ ]:
vocabulary = None

In [ ]:
with open('vocabulary.pickle', "rb") as fp:
            vocabulary = pickle.load(fp)

In [ ]:
class TF_IDF:

    def __init__(self, vocabulary):
        self.vectorizer = CountVectorizer(binary=True, vocabulary=vocabulary)
        self.vectors = None
        self.words = None
        self.dense_vectors_df = None

    def fit_transform_vectorizer(self, dataset):
        self.vectors = self.vectorizer.fit_transform(list(map(lambda doc: ' '.join(doc), dataset)))
        self.words = self.vectorizer.get_feature_names_out()
        dense_vectors = self.vectors.todense().tolist()
        self.dense_vectors_df = pd.DataFrame(dense_vectors, columns=self.words)

    def predict(self, query, dataset, k):
        query = ' '.join(Preprocessor(stopwords_path='stopwords.txt').preprocess(query))
        query_transform = self.vectorizer.transform([query]).todense().tolist()[0]
        dense_vectors = self.dense_vectors_df.values.tolist()
        df_cosine_sim = list(map(lambda doc: self.cosine_sim(query_transform, doc), dense_vectors))
        self.dense_vectors_df['query_sim'] = df_cosine_sim
        indices = self.dense_vectors_df.nlargest(k, 'query_sim').index
        self.dense_vectors_df = self.dense_vectors_df.drop(columns=['query_sim'])
        return dataset.iloc[indices]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "wb") as file:
            pickle.dump(self.vectors.todense().tolist(), file)

    def load_TF_IDF_model(self, path="TF_IDF_model.pickle"):
        with open(path, "rb") as file:
            return pickle.load(file)

    def prepare(self, dataset, mode, save=False):
        model = None
        if mode == 'train':
            self.fit_transform_vectorizer(dataset)
            model = self
        if mode == 'load':
            model = self.load_TF_IDF_model()
        if save:
            self.save_TF_IDF_model()
        return model

In [ ]:
TF_IDF_model = TF_IDF(vocabulary)

In [ ]:
TF_IDF_model.fit_transform_vectorizer(preprocessed_texts[10000:11000])

In [ ]:
TF_IDF_model.fit_transform_vectorizer(preprocessed_texts[1000:2000])

In [ ]:
TF_IDF_model.dense_vectors_df

In [ ]:
vectors = TF_IDF_model.vectors.todense().tolist()

In [ ]:
vectors = np.array(vectors)

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ١. مدل مبتنی بر روش boolean
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
    برای ایجاد این مدل، از کلاس CountVectorizer کتابخانه sklearn استفاده می‌کنیم تا به ازای هر کلمه مشخص شود که در کدام داکیومنت‌ها آمده است. توجه کنید که امکان سرچ با استفاده از عملگرهای or و not نیز پیاده‌سازی شده است. کلمات موجود در یک زیرکوئری همگی رابطه‌ی and را دارند. یک لیست از زیرکوئری‌ها نقش or را ایفا می‌کند. در نهایت، برای exclude کردن یک عبارت از نتایج جست و جو، کافی است در ابتدای عبارت مد نظر کاراکتر "-" گذاشته شود تا به عنوان not عمل کند.
    </p>
</div>

<div>
    <h4 style='direction:rtl;text-align:justify;'>
        ٤. مدل مبتنی بر میانگین وزن‌دار بردارهای تعبیه، مانند FastText
    </h4>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در این بخش، از کتاب‌خانه‌ی fasttext استفاده می‌کنیم و با استفاده از روش skipgram و یک فایل شامل داده‌های موجود اخبار، مدل را به صورت unsupervised به یادگیری وامی‌داریم. پارامترهای استفاده شده‌ی دیگر هم در کد مشخصند؛ به طور مثال، کمینه و بیشینه n را برابر ۲ و ۵ در نظر گرفته‌ایم. پس از انجام یادگیری، میانگین امبدینگ‌ها را در یک آرایه‌ی نامپای ذخیره می‌کنیم. تمام این کارها با صدا زدن متد prepare با حالت train
        رخ می‌دهند. همچنین می‌توانیم این متد را با حالت load صدا بزنیم تا یک مدل ذخیره شده را از فایل بازیابی کنیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        علاوه بر این، می‌توانیم با استفاده از متدهای save_FastText_model و load_FastText_model این مدل و جزئیاتش را ذخیره یا بازیابی کنیم. البته اگر بخواهیم عمل ذخیره را بلافاصله پس از پایان یادگیری انجام دهیم هم می‌توانیم مقدار True را به آرگومان save در متد prepare بدهیم.
    </p>
    <p style='direction:rtl;text-align:justify;'>
        پس از پایان یادگیری یا بازیابی مدل یادگرفته شده از فایل ذخیره شده، می‌توانیم با استفاده از متد predict به تعداد دلخواه خبر مرتبط به درخواستمان را به دست بیاوریم. برای این کار، مدلمان بردارهای مربوط به درخواست را به دست می‌آورد و با محاسبه‌ی کسینوس‌ها و به دست آوردن میزان شباهت بین درخواست و خبرهای موجود، شبیه‌ترین خبرها را خروجی می‌دهد.
    </p>
</div>

In [ ]:
class FastText:

    def __init__(self, preprocessor=None, method='skipgram'):
        self.method = method
        self.mean_embed = []
        self.model = None
        self.preprocessor = preprocessor

    def train(self, texts):
        with open('FastText_train.txt', 'w', encoding='utf-8') as file:
            file.write('\n'.join(list(map(lambda doc: ' '.join(doc), texts))))
        self.model = fasttext.train_unsupervised('FastText_train.txt', self.method, minn=2, maxn=5, wordNgrams=10)
        os.remove('FastText_train.txt')
        self.mean_embed = list(map(lambda doc:
                                   np.mean(list(map(lambda word:
                                                    self.model.get_word_vector(word), doc)), axis=0), texts))
        self.mean_embed = np.array(self.mean_embed)

    def predict(self, query, dataset, k):
        if self.preprocessor:
            query = self.preprocessor.preprocess(query)
        if type(query) == str:
            query = query.split()
        query_embed = np.mean(list(map(lambda word: self.model.get_word_vector(word), query)), axis=0)
        dataset_sim = np.array(list(map(lambda doc: self.cosine_sim(query_embed, doc), self.mean_embed)))
        idx = np.argsort(-dataset_sim)
        return dataset.iloc[list(idx[:k])]

    def cosine_sim(self, query, doc):
        return np.dot(query, doc) / (np.linalg.norm(query) * np.linalg.norm(doc))

    def save_FastText_model(self, path='FastText_model.bin'):
        self.model.save_model(path)
        np.save('FastText_mean_embed.npy', self.mean_embed)

    def load_FastText_model(self, path="FastText_model.bin"):
        self.model = fasttext.load_model(path)
        self.mean_embed = np.load('FastText_mean_embed.npy')

    def prepare(self, dataset, mode, save=False):
        if mode == 'train':
            self.train(dataset)
        if mode == 'load':
            self.load_FastText_model()
        if save:
            self.save_FastText_model()

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، یک instance از مدل FastText ایجاد کرده و شروع به یادگیری داده‌ها و یا خواندن مدل ذخیره شده از روی فایل می‌کنیم.
    </p>
</div>

In [ ]:
FastText_model = FastText()
FastText_model.prepare(preprocessed_texts, mode='train', save=True)

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        با اجرای قطعه کد زیر، نتیجه‌ی پردازش کوئری‌های مشخص شده را مشاهده می‌کنیم.
    </p>
</div>

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        در نهایت، سرچ بر اساس عنوان خبر را پیاده سازی می‌کنیم. این سرچ نیازی به استفاده از مدل‌های زبانی ندارد و صرفاً مانند یک فیلتر عمل می‌کند.
    </p>
</div>

In [ ]:
CATEGORIES_CLASSES = {
    'Politics': 0,
    'World': 1,
    'Economy': 2,
    'Society': 3,
    'City': 4,
    'Sport': 5,
    'Science': 6,
    'Culture': 7,
    'IT': 8,
    'LifeSkills': 9,
}

In [ ]:
dataset['category_code'] = dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = dataset['category_code'].to_numpy()
labels.shape

In [ ]:
mini_dataset['category_code'] = mini_dataset['category'].apply(lambda x: CATEGORIES_CLASSES[x])
labels = mini_dataset['category_code'].to_numpy()
labels.shape

In [ ]:
perm = np.random.permutation(len(labels))
perm

In [ ]:
train_size = int(0.85 * len(labels))
train_size

In [ ]:
train_vectors = vectors[perm[:train_size]]
train_labels = labels[perm[:train_size]]
test_vectors = vectors[perm[train_size:]]
test_labels = labels[perm[train_size:]]

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(max_iter=1000, random_state=0)

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

In [ ]:
clf.fit(train_vectors, train_labels)

In [ ]:
clf.score(test_vectors, test_labels)

In [ ]:
y = clf.predict(test_vectors)
y

In [ ]:
from sklearn.metrics import f1_score

f1_score(test_labels, y, labels=np.arange(10), average='macro')

In [ ]:
clf.predict(vectors[40000:40005, :])

<div dir="auto" align="justify">
    <p style='direction:rtl;text-align:justify;'>
        از این که وقت ارزشمند خود را برای بررسی این نوتبوک صرف کرده‌اید، صمیمانه سپاسگزاریم. (:
    </p>
</div>

In [ ]:
from sklearn.cluster import KMeans

k_means = KMeans(n_clusters=10, random_state=0)

In [ ]:
k_means.fit(vectors)

In [ ]:
y = k_means.labels_
y

In [ ]:
from  matplotlib import pyplot as plt


plt.scatter(np.arange(100), y[:100])
plt.show()